# Calculator Bool Exam

|Entrada| Salida |
|:-:|:-:|
|x := 2;||
|print x;|Resultado es verdadero|
|y := 0 and x; | |
|print nor y;| Resultado es verdadero|
|print x and not y;| Resultado es verdader|
|print not (x and not y) ; | Resultado es falso |
| x:= not x; | |
|z:= true or not (x and not y); ||
|w := (3 > 5)  and (z < 4);||
|print 4 == (w < 3);| Resultado es falso |



## Grammar and Traduction scheme

```
input -> `print` expr {escribe(expr.s)};
input -> asig;

asig -> ID `=` expr {table[ID.lexval] = expr.s}

expr -> expr_1 `or` exprOR {expr_1.s || exprOR.s}
expr -> exprOR {expr.s = exprOR.s}

exprOR -> exprOR_1 `and` exprAND {exprOR_1.s && exprAND.s}
exprOR -> exprAND {exprOR.s = exprAND.s}

exprAND -> boolean {exprAND.s = boolean.s}

boolean -> not boolean_1 {boolean.s = !boolean_1.s}
boolean -> CBOOLEAN {boolean.s = CBOOLEAN}
boolean -> ID {boolean.s = table[ID.Lexval]}
boolean -> (expr) {boolean.s = expr.s}
```

## L-recursive to R-recursive

```
input -> `print` expr {escribe(expr.s)};
input -> asig;
asig -> ID `=` expr {table[ID.lexval] = expr.s}

expr -> exprOR expr'
expr' -> `||` exprOR 
expr' -> e

exprOR -> exprAND exprOR'
exprOR' -> `and` exprAND expOR'
exprOR' -> e

boolean -> not boolean
exprAnd -> boolean
boolean -> CBOOL
boolean -> ID
boolean -> `(`expr`)` 

```

## First and Second Set

### First set

|First | Elementos|
|:-:|:-:|
|boolean| {CBOOL, 'ID', '(', 'not'} |
|exprAND| {CBOOL, 'ID', '(', 'not'} |
|exprOR | {CBOOL, 'ID', '(', 'not'} |
|exprOR' | { 'and' , e, } |
|expr' | {`or`, e} | 
|expr | {CBOOL, 'ID', '(', 'not'} |
| asig | {'ID'} |
| input | {'ID', 'print'} |


### Second set

|Second| Elementos|
|:-:|:-:|
|input |{$}|
|asig|';'|
|expr|';'|
|expr'|';'|
|exprOR|';' |